## End-to-End model for classification emotions from audio files.

This is a follow up to the preliminary file 'model_first', where data processing and base models were made.

The base model evaluates at 52.8% accuracy.

Let's see if I can improve the performance using and end-to-end deep learning model.

I will first start with a Convolutional Neural Network which works with spectrograms of the audio files.

In [2]:
### IMPORTS 

## Audio libraries (install if needed)

import noisereduce as nr # for noise reduction
import librosa # for feature extraction
import pydub # for normalising audio

import os
import numpy as np
import time

from pydub import AudioSegment
import matplotlib.pyplot as plt
import librosa.display
from pydub import effects

cwd = os.getcwd()
database = cwd + '\dataset\MSP-IMPROV'

In [5]:
max_len = 338368

In [9]:
labels = []
audios = []
spectrograms = []
emotions = {'S':'sad', 'A': 'angry', 'N':'neutral', 'H':'happy', 'X':'X', 'O': 'other'}
max_len = 338368
start = time.perf_counter()


pat = cwd + '\dataset\Evalution.txt'
with open(pat) as f:
    contents = f.read()
    lines = contents.split(';')

for subdir, dirs, files in os.walk(database):
    for file in files: 
        # Preprocessing
        process = file.split('-')[-2]
        if process in ['R', 'T']:
            
            y, sr = librosa.load(path = os.path.join(subdir,file), sr = None)
            audio = AudioSegment.from_file(os.path.join(subdir,file)) 

            normalized = effects.normalize(audio)
            full_normalized = (np.array(audio.get_array_of_samples(), dtype = 'float32')/32768*1000)
            full_normalized = librosa.core.resample(full_normalized, orig_sr=22050, target_sr=22050, res_type='kaiser_best')
            trim, ix = librosa.effects.trim(full_normalized, top_db=25,frame_length=256, hop_length=64)
            full_normalized = np.pad(trim, (0, max_len - len(trim)),)
            fin = nr.reduce_noise(full_normalized, sr=sr,)

            # Create spectrogram:
            stft_fin = librosa.stft(fin)
            spectrogram = librosa.amplitude_to_db(abs(stft_fin)) 
            
            file = '\n\nUTD'+file[3:-3]+'avi'
            if file == '\n\nUTD-IMPROV-S01A-F02-R-FF01.avi':
                file = '\nUTD-IMPROV-S01A-F02-R-FF01.avi'
            i = lines.index(file)+1
            emotion = emotions[lines[i][1:]]
            labels.append(emotion)
            audios.append(fin)
            spectrograms.append(spectrogram)

end = time.perf_counter()
print(f"Total time taken: {(end - start)/60:0.3f} minutes")

Total time taken: 8.151 minutes


In [12]:
np.save('specs.npy', spectrograms)
np.save('audios.npy', audios)
np.save('labels.npy', labels)


In [15]:
## Save them so I can load instead of preprocessing every time

In [4]:
specs = np.load('specs.npy')
audios = np.load('audios.npy')
labels = np.load('labels.npy')

In [ ]:
# Move to colab.